In [22]:
print('Hello LangChain')

Hello LangChain


In [23]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [24]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [25]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001F7822BB200> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F78283B2F0> root_client=<openai.OpenAI object at 0x000001F78283BFE0> root_async_client=<openai.AsyncOpenAI object at 0x000001F78283B950> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [ ]:
try:    # llm 호출
    response = llm.invoke(prompt_text) 
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [27]:
from langchain_core.prompts import PromptTemplate


prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 한글로 설명해주세요.
    """)
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    You are an expert in AI Expert. Answer the question. \n    <Question>: {input}에 대해 쉽게 한글로 설명해주세요.\n    ')

In [28]:
# chain 연결 (LCEL)
chain = prompt | llm

### LCEL
* Prompt + LLM + OuterParser를 Chain으로 연결하기

In [29]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL)
output_parser = StrOutputParser()
chain2 = prompt | llm | output_parser

In [30]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 방식과 유사합니다. 컴퓨터가 데이터를 통해 스스로 규칙을 찾고, 그 규칙을 바탕으로 미래의 데이터를 예측하거나 분류하는 것입니다.

쉽게 말해, 인공지능 모델이 학습하는 과정을 다음과 같이 비유할 수 있습니다.

1. **데이터 수집**: 학생이 공부를 시작할 때, 책이나 자료를 많이 모아야 하듯이, 인공지능 모델도 학습을 위해 많은 데이터를 수집합니다.

2. **데이터 준비**: 수집한 데이터를 정리하고, 필요한 경우 데이터를 변환하거나 가공합니다. 이는 학생이 공부 자료를 정리하고 이해하기 쉽게 만드는 과정과 비슷합니다.

3. **모델 훈련**: 이 데이터를 바탕으로 인공지능 모델이 스스로 규칙이나 패턴을 찾습니다. 예를 들어, 고양이와 강아지의 사진을 많이 보여주면서 '이것은 고양이야', '이것은 강아지야'라고 알려주는 겁니다. 모델은 이 과정을 통해 고양이와 강아지를 구별하는 규칙을 스스로 터득합니다.

4. **평가**: 훈련이 끝난 후, 모델이 얼마나 잘 학습했는지 평가합니다. 새로운 사진들을 모델에 보여주고, 모델이 올바르게 분류하는지 확인하는 과정입니다.

5. **반복**: 모델의 성능이 만족스럽지 않다면, 더 많은 데이터를 수집하거나, 모델의 구조를 바꾸는 등 다양한 방법으로 학습 과정을 반복합니다.

이를 통해 인공지능 모델은 주어진 데이터를 기반으로 하여, 새로운 데이터에 대해서도 정확한 예측이나 분류를 할 수 있게 됩니다.

예를 들어, 음성 인식 인공지능 모델을 개발한다고 가정해 봅시다. 이 모델에게 수많은 사람들의 목소리를 녹음한 데이터를 제공하고, 그것이 어떤 단어인지, 어떤 문장인지를 학습시킵니다. 그러면 이 모델은 새로운 목소리를 듣고 그 내용을 예측할 수 있게 됩니다.

이처럼 인공지능 모델의 학습은 데이터로부터 패턴을 발견하고, 그 패턴을 바탕으로 미래의 데이터를 이해하는 과정이라고 할 수

In [31]:
# chain 호출
try:
    result = chain2.invoke({"input": "LangChain의 Product(제품)에는 어떤 것들이 있나요?"})
    print(type(result))
    print(result) # output parser라서 스트링으로 바로 나옴
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 인공지능(AI) 기술을 기반으로 하는 플랫폼으로, 다양한 제품과 서비스를 제공하고 있습니다. LangChain의 주요 제품은 다음과 같습니다.

1. **LangChain 플랫폼**: LangChain은 개발자들이 AI 기반 애플리케이션을 쉽게 구축할 수 있도록 지원하는 플랫폼입니다. 이 플랫폼은 자연어 처리(NLP), 기계 학습(ML), 딥러닝 등 다양한 AI 기술을 통합하여 제공합니다.

2. **LangChain API**: LangChain API는 개발자들이 자사의 애플리케이션에 AI 기능을 통합할 수 있도록 지원하는 API입니다. 이 API를 통해 개발자들은 자연어 처리, 이미지 인식, 음성 인식 등 다양한 AI 기능을 사용할 수 있습니다.

3. **LangChain Studio**: LangChain Studio는 AI 모델을 개발하고 훈련시키는 데 사용되는 개발 환경입니다. 이 환경은 개발자들이 AI 모델을 쉽게 생성, 훈련, 배포할 수 있도록 지원합니다.

4. **LangChain 모델**: LangChain은 다양한 사전 훈련된 AI 모델을 제공합니다. 이 모델들은 자연어 처리, 컴퓨터 비전, 음성 인식 등 다양한 분야에서 활용될 수 있습니다.

5. **LangChain 에이전트**: LangChain 에이전트는 사용자가 자연어로 요청을 하면 그에 따라 작업을 수행하는 AI 에이전트입니다. 사용자는 에이전트를 통해 다양한 작업을 요청할 수 있으며, 에이전트는 그에 따라 적절한 답변을 제공합니다.

이러한 제품과 서비스를 통해 LangChain은 개발자와 기업이 AI 기술을 쉽게 활용할 수 있도록 지원하고 있습니다.


### Runnable의 stream() 함수 호출

In [ ]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})

    # 스트리밍 출력
    # print(answer)

    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")



### Multi Chain
* 첫번째 Chain의 출력이, 두번째 Chain의 입력이 된다.
* 두개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    # api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [ ]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "Drama"})
    print(response)
except Exception as e:
    print(f"오류 발생: {e}")

### PromptTemplate 여러개 연결하기

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

# OpenAI 모델 사용
llm = ChatOpenAI(
    # api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llama-4", "count": 4}
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions] # q가 dict 타입 **의 의미는 dict가 넘어갈 때 사용
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

In [ ]:
# OpenAI 모델 사용
llm = ChatOpenAI(
    # api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt)
    print(response.content)

### ChatPromptTemplate
* SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePrompt Template 사용

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {topic} 전문가 입니다. 명확하고 자세하게 설명해 주세요."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝은 무엇인가요?")

# LLM 호출
# OpenAI 모델 사용
llm = ChatOpenAI(
    # api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

### FewShotPromptTemlate
* 예시를 제공하는 프롬프트

In [46]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
model = ChatOpenAI(
    # api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성
1. **수성**: 태양과 가장 가까운 행성으로, 매우 작고 온도가 극심하게 변합니다.
2. **금성**: 밝고 뜨거운 행성으로, 강한 온실 효과로 표면 온도가 높습니다.
3. **지구**: 생명체가 살고 있는 행성으로, 물과 대기가 있습니다.
4. **화성**: 붉은 행성으로, 과거에 물이 있었다고 추정되며 탐사 로봇이 자주 방문합니다.
5. **목성**: 태양계에서 가장 큰 행성으로, 가스 행성입니다.
6. **토성**: 아름다운 고리를 가진 가스 행성입니다.
7. **천왕성**: 빙하 행성으로, 자전축이 기울어져 있습니다.
8. **해왕성**: 가장 먼 행성으로, 강한 바람과 추운 날씨를 가지고 있습니다.
